# **0. 구글 드라이브 임포트**

In [1]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [2]:
# 드라이브 위치 확인
%cd /content/gdrive/MyDrive/Colab Notebooks/PITer

/content/gdrive/MyDrive/Colab Notebooks/PITer


In [3]:
# How to check HF access token: hugging face -> select account at top right -> settings -> access token
import huggingface_hub
huggingface_hub.login()

# **1. Install and Import**

In [4]:
!pip install datasets
!pip install transformers accelerate bitsandbytes peft trl
!pip install --upgrade transformers trl
!pip install chardet
!pip install jsonlines
!pip install torch
!pip install gguf

# !pip install --upgrade torch torchvision torchaudio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 33.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.9/310.9 kB 28.4 MB/s eta 0:00:0

In [5]:
import torch
import pandas as pd
import numpy as np
import warnings
import time
import chardet

import json
import jsonlines

from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    DataCollatorForLanguageModeling,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
    logging,
)

from peft import LoraConfig, get_peft_model
from trl import SFTTrainer
from huggingface_hub import notebook_login
from datasets import load_dataset

import sys
import logging
import datasets
import transformers

logger = logging.getLogger(__name__)

In [ ]:
# 모델 로딩 (초기에 다운로드 받을 때 [구글 드라이브에 로컬 모델 없을 때!])
model_id = "LGAI-EXAONE/EXAONE-3.0-7.8B-Instruct"
# model_id = "origin"
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    trust_remote_code=True,
    device_map="auto"
)

tokenizer = AutoTokenizer.from_pretrained(model_id)

In [ ]:
# 로컬 드라이브 (구글)에 저장할때 쓰는 코드 한 번만 실행!

from transformers import AutoModelForCausalLM, AutoTokenizer

# 모델 ID 설정
model_id = "LGAI-EXAONE/EXAONE-3.0-7.8B-Instruct"

# 모델과 토크나이저 로드
model = AutoModelForCausalLM.from_pretrained(model_id)
tokenizer = AutoTokenizer.from_pretrained(model_id)

# 로컬에 저장할 경로 설정
save_directory = "./origin"

# 모델과 토크나이저 저장
model.save_pretrained(save_directory)
tokenizer.save_pretrained(save_directory)


In [6]:
# Google Drive에 저장된 모델 로드
from transformers import AutoModelForCausalLM, AutoTokenizer
from google.colab import drive

# 저장된 모델 경로 (Google Drive의 origin 폴더)
model_path = "/content/gdrive/MyDrive/Colab Notebooks/PITer/origin"

# 모델과 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path)

# 모델 로드 확인
print("Model and tokenizer successfully loaded from:", model_path)

The repository for /content/gdrive/MyDrive/Colab Notebooks/PITer/origin contains custom code which must be executed to correctly load the model. You can inspect the repository content at https://hf.co//content/gdrive/MyDrive/Colab Notebooks/PITer/origin.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


configuration_exaone.py:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/LGAI-EXAONE/EXAONE-3.0-7.8B-Instruct:
- configuration_exaone.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


The repository for /content/gdrive/MyDrive/Colab Notebooks/PITer/origin contains custom code which must be executed to correctly load the model. You can inspect the repository content at https://hf.co//content/gdrive/MyDrive/Colab Notebooks/PITer/origin.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


modeling_exaone.py:   0%|          | 0.00/81.1k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/LGAI-EXAONE/EXAONE-3.0-7.8B-Instruct:
- modeling_exaone.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

Model and tokenizer successfully loaded from: /content/gdrive/MyDrive/Colab Notebooks/PITer/origin


# **2. Data Preprecessing and Tokenuzer**

In [7]:
pwd

'/content/gdrive/MyDrive/Colab Notebooks/PITer'

In [7]:
# 데이터셋 패스 정하기
dataPath = "/content/gdrive/MyDrive/Colab Notebooks/PITer/datasets/modifyData.csv"

In [8]:
dataPath = load_dataset("csv", data_files=dataPath)

Generating train split: 0 examples [00:00, ? examples/s]

In [9]:
train_data = dataPath

In [32]:
# train_dataset = train_data['train']

In [10]:
train_data

DatasetDict({
    train: Dataset({
        features: ['0'],
        num_rows: 5000
    })
})

In [23]:
from datasets import DatasetDict

# 기존 데이터셋에서 'train' 부분을 추출
train_dataset = train_data['train']

# 무작위로 5,000개의 샘플 선택
reduced_train_dataset = train_dataset.shuffle(seed=24).select(range(5000))

# 새로운 DatasetDict 생성
reduced_dataset_dict = DatasetDict({'train': reduced_train_dataset})

# 결과 확인
print(reduced_dataset_dict)


DatasetDict({
    train: Dataset({
        features: ['0'],
        num_rows: 5000
    })
})


In [25]:
# CSV 파일로 저장
reduced_train_dataset.to_csv('modifyData.csv', index=False)

Creating CSV from Arrow format:   0%|          | 0/5 [00:00<?, ?ba/s]

1118407

In [26]:
# 저장된 CSV 파일 확인
import pandas as pd

df = pd.read_csv('modifyData.csv')
print(df.head())

                                                   0
0  ### Instruction: 양념치킨하고 후라이드 치킨을 반반씩 시켜서 후라이드 ...
1  ### Instruction: 장래희망이 뭐냐니깐! 이 문장을 정중한 말투로 바꿔줘...
2  ### Instruction: 당연ㅋㅋ 누가 반려동물 잡아먹음? 이 문장을 정중한 ...
3  ### Instruction: ㅇㅇ 항상 빨간 건 아니고 인터뷰 맞춰서 염색한 거임...
4  ### Instruction: 너 밥 먹고 나서 양치 안하지? 이 문장을 정중한 말...


In [11]:
# 마지막 데이터 확인
last_data = train_data['train'][-1]
print(last_data)
info_data = train_data['train'][3]
print(info_data)
info_data2 = train_data['train'][3240]
print(info_data2)

{'0': '### Instruction: 난 지금 뮤지컬 보고 오는 길임 이 문장을 정중한 말투로 바꿔줘.\n\n### Response: 저는 지금 뮤지커을 보고 돌아오는 길입니다.[|endofturn|]'}
{'0': '### Instruction: ㅇㅇ 항상 빨간 건 아니고 인터뷰 맞춰서 염색한 거임 이 문장을 정중한 말투로 바꿔줘.\n\n### Response: 네, 항상 빨간 건 아니고 인터뷰에 맞춰서 염색한 것입니다.[|endofturn|]'}
{'0': '### Instruction: 허나 짐의 점심을 조금 주겠소. 새우 부리토요. 이 문장을 정중한 말투로 바꿔줘.\n\n### Response: 하지만 제 점심을 조금 드릴게요. 새우 부리토입니다.[|endofturn|]'}


In [12]:
# 토큰화 정의 및 설정
tokenizer = AutoTokenizer.from_pretrained(
              "LGAI-EXAONE/EXAONE-3.0-7.8B-Instruct",
              trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" # 패딩이 입력 시퀀스의 오른쪽에 추가되도록 설정

tokenizer_config.json:   0%|          | 0.00/70.7k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.93M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.22M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/4.96M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/563 [00:00<?, ?B/s]

In [ ]:
# 데이터셋 토큰화
'''
확인 후 설정

def tokenize_function(example):
    # 'Instruction'과 'Response'를 결합
    text = example['0']
    # 토큰화
    return tokenizer(text, padding="max_length", truncation=True, max_length=512)

# 데이터셋 토큰화 적용
tokenized_dataset = dataset.map(tokenize_function, batched=True)
'''


# **3. Setting Configs**

In [13]:
# 현재 사용 중인 GPU의 주요 아키텍처 버전을 반환 8버전 이상 시 bfloat16 활용
if torch.cuda.get_device_capability()[0] >= 8:
    !pip install -qqq flash-attn
    attn_implementation = "flash_attention_2"
    torch_dtype = torch.bfloat16
else:
    attn_implementation = "eager"
    torch_dtype = torch.float16

# BitsAndBytesConfig 객체활용 양자화 설정
# 모델을 4비트 양자화하여 로드
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch_dtype,
    bnb_4bit_use_double_quant=False,
)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 75.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [12]:
# LoRA 설정
peft_params = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
)

# **4 파인튜닝 설정 및 시작**

In [20]:
import gc
import torch

# 가비지 컬렉션 수행
gc.collect()
# PyTorch 캐시 비우기
torch.cuda.empty_cache()

In [13]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

In [31]:
!ps -ef

UID          PID    PPID  C STIME TTY          TIME CMD
root           1       0  0 18:14 ?        00:00:00 /sbin/docker-init -- /datalab/run.sh
root           8       1  0 18:14 ?        00:00:03 /tools/node/bin/node /datalab/web/app.js
root          10       8  0 18:15 ?        00:00:00 /bin/bash -e /usr/local/colab/bin/oom_monitor.sh
root          12       1  0 18:15 ?        00:00:00 /bin/bash -e /datalab/run.sh
root          13      12  0 18:15 ?        00:00:00 /usr/colab/bin/kernel_manager_proxy --listen_por
root          41       0  0 18:15 ?        00:00:00 tail -n +0 -F /root/.config/Google/DriveFS/Logs/
root          50       0  0 18:15 ?        00:00:00 tail -n +0 -F /root/.config/Google/DriveFS/Logs/
root          87       8  0 18:15 ?        00:00:10 [python3] <defunct>
root          88       8  0 18:15 ?        00:00:00 python3 /usr/local/bin/colab-fileshim.py
root         133       8  0 18:15 ?        00:00:04 /usr/bin/python3 /usr/local/bin/jupyter-notebook
root       

In [42]:
!ps -ef | grep jupyter

root         133       8  0 18:15 ?        00:00:04 /usr/bin/python3 /usr/local/bin/jupyter-notebook
root        8415    1619  0 18:44 ?        00:00:00 /bin/bash -c ps -ef | grep jupyter
root        8417    8415  0 18:44 ?        00:00:00 grep jupyter


In [47]:
!ps -ef | grep cuda

root        8675    1619  0 18:45 ?        00:00:00 /bin/bash -c ps -ef | grep cuda
root        8677    8675  0 18:45 ?        00:00:00 grep cuda


In [17]:
!kill -9 [74888]

/bin/bash: line 1: kill: [74888]: arguments must be process or job IDs


In [55]:
# nvidia
# this is terminal command
!nvidia-smi

Sun Nov 24 18:48:02 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0              49W / 400W |  30831MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [14]:
from trl import SFTConfig, SFTTrainer

# SFTConfig 설정
sft_config = SFTConfig(
    output_dir="./results",
    dataset_text_field="0",  # 데이터셋의 텍스트 필드 이름
    dataset_num_proc=2,      # 데이터셋 처리에 사용할 프로세스 수
    max_seq_length=512,      # 시퀀스의 최대 길이
    num_train_epochs=3,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    optim="paged_adamw_8bit",
    save_steps=75,
    learning_rate=2e-5,
    warmup_steps=250,
    weight_decay=0.005,
    fp16=True,
    bf16=False,
    max_grad_norm=1.0,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="constant",
    report_to="none",
    gradient_checkpointing=True  # Gradient Checkpointing 활성화
)

# SFTTrainer 초기화
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=train_data['train'],
    args=sft_config,
)

Map (num_proc=2):   0%|          | 0/5000 [00:00<?, ? examples/s]

In [22]:
# 학습 시작
trainer.train()

OutOfMemoryError: CUDA out of memory. Tried to allocate 112.00 MiB. GPU 0 has a total capacity of 39.56 GiB of which 96.81 MiB is free. Process 74888 has 39.46 GiB memory in use. Of the allocated memory 38.90 GiB is allocated by PyTorch, and 63.81 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

# **5. 허깅페이스 모델 업로드**

In [ ]:
# 허깅페이스로 업로드
repoId = "LINKER98/PITerAI"

# 트레이너로 학습 한 모델
#trainer.push_to_hub(repoId)

# 직접 모델로 추출 후 업로드 코드
model.push_to_hub(repoId)
tokenizer.push_to_hub(repoId)

# **6. 양자화 및 재배포**

# **7. Loading Pre-trained Model + Test**

In [ ]:
# 프롬프트에 입력에 대한 결과 출력 함수 선언

def text_generation(prompt):

  messages = [
    {"role": "system",
     "content": "You are EXAONE model from LG AI Research, a helpful assistant."},
    {"role": "user", "content": prompt}
  ]

  input_ids = tokenizer.apply_chat_template(
      messages,
      tokenize=True,
      add_generation_prompt=True,
      return_tensors="pt"
  )

  output = model.generate(
      input_ids.to("cuda"),
      eos_token_id=tokenizer.eos_token_id,
      max_new_tokens=512
  )

  print(tokenizer.decode(output[0], skip_special_tokens=True))

In [ ]:
text_generation("ㅎㅇ 요즘 뭐하고 지내냐? 이 문장을 정중한 말투로 바꿔줘.")

[|system|]You are EXAONE model from LG AI Research, a helpful assistant.
[|user|]ᄒᄋ 요즘 뭐하고 지내냐? 이 문장을 정중한 말투로 바꿔줘.
[|assistant|]안녕하세요. 요즘 어떻게 지내시나요?


In [ ]:
text_generation("별로 상황이 좋지는 않아.. 지난달에 은행에서 잘렸거든 ㅠ 이 문장을 정중한 말투로 바꿔줘.")

[|system|]You are EXAONE model from LG AI Research, a helpful assistant.
[|user|]별로 상황이 좋지는 않아.. 지난달에 은행에서 잘렸거든 ᅲ 이 문장을 정중한 말투로 바꿔줘.
[|assistant|]상황이 별로 좋지는 않습니다. 지난 달에 은행에서 잘렸어요.


In [ ]:
text_generation("보험계리사의 업무에 대해서 알려줘.")

[|system|]You are EXAONE model from LG AI Research, a helpful assistant.
[|user|]보험계리사의 업무에 대해서 알려줘.
[|assistant|]보험계리사는 보험회사 및 보험대리점, 보험중개사무소 등에서 보험계리 업무를 수행하고 있습니다. 보험계리 업무는 보험회사의 재무건전성을 확보하고, 보험계약자, 피보험자 및 보험수익자 등 보험계약자의 이익을 보호하기 위하여 보험상품의 가격(보험료)을 산출하고, 보험계약자의 보험금 지급 청구 시 손해의 사실 여부를 조사하고 보험금을 지급하는 업무 등을 수행하는 것입니다.<eos>

보험계리사는 보험계리 업무를 수행할 수 있는 전문자격자로, 이 자격을 취득하기 위해서는 금융감독원이 시행하는 보험계리사 시험에 합격하여야 합니다.<eos>

보험계리사 시험은 1차 시험과 2차 시험으로 구분되어 있으며, 1차 시험에 합격한 후 2차 시험에 합격하여야 합니다. 시험은 매년 1회 실시되고 있으며, 1차 시험은 객관식 5과목으로 구성되어 있고, 2차 시험은 주관식 3과목으로 구성되어 있습니다.<eos>

보험계리사 시험의 합격기준은 1차 시험의 경우 각 과목의 만점의 60% 이상을 득점하고, 각 과목을 합산하여 60점 이상을 득점한 자 중에서 선발예정인원의 10배를 합격자로 선정합니다. 2차 시험의 경우에는 만점의 40% 이상을 득점한 자 중에서 선발예정인원의 5배를 합격자로 선정합니다.<eos>

보험계리사 시험의 응시자격에는 제한이 없으므로 누구나 응시할 수 있습니다.<eos>

시험과목 및 합격기준 등에 대한 자세한 사항은 금융감독원 홈페이지에서 확인할 수 있습니다.<eos>

보험계리사에 대한 자세한 사항은 금융감독원 홈페이지(www.fss.go.kr)에서 확인할 수 있습니다.<eos>


# **(안쓰는 것)3. Data Preprecessing**

In [ ]:
dataPath = "/content/data/smilestyle_dataset.tsv"

# Open the file in binary mode
with open(dataPath, 'rb') as f:
    # Detect the encoding
    result = chardet.detect(f.read())

dataset = pd.read_csv(dataPath, encoding=result['encoding'], sep="\t", on_bad_lines="warn")

In [ ]:
save_dataset_to_csv(new_dataset, "/content/data/modified_traindata.csv")

In [ ]:
EOS_TOKEN = tokenizer.eos_token

In [ ]:
print(dataset['translator'][3704])

nan


In [ ]:
def generate_prompt(example):
  output_texts = []
  #appending
  for i in range(len(example['chat'])):
    prompt = f"### Instruction: {example['chat'][i]} 이 문장을 정중한 말투로 바꿔줘.\n\n### Response: {example['formal'][i]}"
    prompt = prompt + EOS_TOKEN
    output_texts.append(prompt)

  for i in range(len(example['choding'])):
    prompt = f"### Instruction: {example['choding'][i]} 이 문장을 정중한 말투로 바꿔줘.\n\n### Response: {example['formal'][i]}"
    prompt = prompt + EOS_TOKEN
    output_texts.append(prompt)

  for i in range(len(example['sosim'])):
    prompt = f"### Instruction: {example['sosim'][i]} 이 문장을 정중한 말투로 바꿔줘.\n\n### Response: {example['formal'][i]}"
    prompt = prompt + EOS_TOKEN
    output_texts.append(prompt)

  for i in range(len(example['translator'])):
    prompt = f"### Instruction: {example['translator'][i]} 이 문장을 정중한 말투로 바꿔줘.\n\n### Response: {example['formal'][i]}"
    prompt = prompt + EOS_TOKEN
    output_texts.append(prompt)

  return output_texts

In [ ]:
new_dataset = generate_prompt(dataset)

In [ ]:
new_dataset

['### Instruction: 하잉ㅋㅋ 나 떼걸룩 6마리 키운다! 이 문장을 정중한 말투로 바꿔줘.\n\n### Response: 안녕하세요. 저는 고양이 6마리 키워요.[|endofturn|]',
 '### Instruction: 엥? 6마리나? 안힘듬?ㅋㅋㅋㅋ 이 문장을 정중한 말투로 바꿔줘.\n\n### Response: 고양이를 6마리나요? 키우는거 안 힘드세요?[|endofturn|]',
 '### Instruction: 내가 고양이 좋아해서 딱히 안힘듬 ㅋㅋㅋ 이 문장을 정중한 말투로 바꿔줘.\n\n### Response: 제가 워낙 고양이를 좋아해서 크게 힘들진 않아요.[|endofturn|]',
 '### Instruction: 가장 나이 먹은 고양이가 몇살이야? 이 문장을 정중한 말투로 바꿔줘.\n\n### Response: 가장 나이가 많은 고양이가 어떻게 돼요?[|endofturn|]',
 '### Instruction: 이제 여섯살이고 갈색고양이임! 이 문장을 정중한 말투로 바꿔줘.\n\n### Response: 여섯 살입니다. 갈색 고양이에요.[|endofturn|]',
 '### Instruction: 그럼 가장 애기인 고양이는 몇살이야? 이 문장을 정중한 말투로 바꿔줘.\n\n### Response: 그럼 가장 어린 고양이가 어떻게 돼요?[|endofturn|]',
 '### Instruction: 작년에 분양 받았는데 1살이야! 이 문장을 정중한 말투로 바꿔줘.\n\n### Response: 한 살입니다. 작년에 분양 받았어요.[|endofturn|]',
 '### Instruction: 냥이들끼리 안 싸움? 이 문장을 정중한 말투로 바꿔줘.\n\n### Response: 그럼 고양이들끼리 안 싸우나요?[|endofturn|]',
 '### Instruction: 우리 일곱은 다 한 가족이라서 싸울 일이 없어 이 문장을 정중한 말투로 바꿔줘.\n\n### Response: 저희 일곱은 다같이 한 가족입니다. 싸우는 일은 없어요.

In [ ]:
print(new_dataset[4000])

### Instruction: ㅎㅇ 요즘 뭐하고 지내냐? 이 문장을 정중한 말투로 바꿔줘.

### Response: 안녕하세요. 요즘은 어떻게 지내시나요?[|endofturn|]


In [ ]:
print(new_dataset[7706])

### Instruction: 별로 상황이 좋지는 않아.. 지난달에 은행에서 잘렸거든 ㅠ 이 문장을 정중한 말투로 바꿔줘.

### Response: 별로 상황이 좋지는 않습니다. 지난 달에 은행에서 잘렸어요.[|endofturn|]


In [ ]:
def save_dataset_to_csv(dataset, file_path):
    """
    주어진 데이터셋을 CSV 파일로 저장합니다.

    매개변수:
    - dataset: 리스트의 리스트 형태로, 각 내부 리스트는 데이터의 한 행을 나타냅니다.
    - file_path: 저장할 CSV 파일의 경로와 이름을 포함한 문자열입니다.

    반환값:
    - 없음
    """
    # 데이터셋을 DataFrame으로 변환
    df = pd.DataFrame(dataset)

    # DataFrame을 CSV 파일로 저장
    df.to_csv(file_path, index=False, encoding='utf-8-sig')

In [ ]:
save_dataset_to_csv(new_dataset, "/content/data/modified_traindata.csv")

In [ ]:
train_data = load_dataset("csv", data_files="/content/data/traindata.csv")

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
pwd

'/content'

In [ ]:
train_data

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 10219
    })
})

In [ ]:
print(train_data['train'][4001]['text'])

### Instruction: 어떤 거 배움?? 이 문장을 정중한 말투로 바꿔줘.

### Response: 어떤 사실들을 배웠나요?[|endofturn|]
